<a href="https://colab.research.google.com/github/esdrasantos/etl-faturas-nubank/blob/main/qui_quadrado_semana_categoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np

In [25]:
import pandas_gbq

# Replace with your project ID and table name
BQ_PROJECT = 'powerbi-drive-458204'
test_table_name = 'powerbi-drive-458204.controle_gastos.fatura_nu_prep'

query = f'SELECT Data,   \
    Categoria,           \
    `Arquivo Origem`,    \
    `Fech Fatura`,       \
    `Semana Compra`,     \
    Dia,                 \
    `Mes Compra`,        \
    ROUND(Valor,2) Valor,\
    Compra               \
FROM `{test_table_name}` ORDER BY Data ASC'

df = pandas_gbq.read_gbq(query, project_id=BQ_PROJECT)

Downloading: 100%|██████████|


In [26]:
df['Valor'] = df['Valor'].astype(float).round(2)

In [41]:
df['Dia'] = df['Dia'].astype(int)

In [42]:
dados = pd.DataFrame(df)
dados['Semana Mes'] = dados['Dia'].apply(lambda x: x // 7 + 1)
dados.head()

,Data,Categoria,Arquivo Origem,Fech Fatura,Semana Compra,Dia,Mes Compra,Valor,Compra,Semana Mes
0,2024-12-01,Transporte,faturas_nu/Nubank_2025-01-11.pdf,2025-01-11,48,1,12,12.99,UBER TRIP,1
1,2024-12-04,Shopping e ecommerce,faturas_nu/Nubank_2025-01-11.pdf,2025-01-11,49,4,12,53.13,AMAZON MARKETPLACE,1
2,2024-12-04,Shopping e ecommerce,faturas_nu/Nubank_2025-01-11.pdf,2025-01-11,49,4,12,53.28,AMAZON MARKETPLACE,1
3,2024-12-04,Shopping e ecommerce,faturas_nu/Nubank_2025-01-11.pdf,2025-01-11,49,4,12,55.20,AMAZON MARKETPLACE,1
4,2024-12-04,Shopping e ecommerce,faturas_nu/Nubank_2025-01-11.pdf,2025-01-11,49,4,12,30.08,MERCADOLIVRE5PRODUTO,1


In [47]:
analise = dados[['Semana Mes', 'Categoria']]
analise.head()

,Semana Mes,Categoria
0,1,Transporte
1,1,Shopping e ecommerce
2,1,Shopping e ecommerce
3,1,Shopping e ecommerce
4,1,Shopping e ecommerce


In [56]:
count = (
    analise.groupby(['Semana Mes','Categoria'])
    .agg(Contagem=('Categoria', 'count'))
    .reset_index())
count.head()

,Semana Mes,Categoria,Contagem
0,1,Apostas,1
1,1,Assinaturas e Servicos,3
2,1,Bar,1
3,1,Beleza,12
4,1,Casa,2


In [55]:
tabela = pd.crosstab(analise['Categoria'], analise['Semana Mes'])
tabela

Semana Mes,1,2,3,4,5
Categoria,,,,,
Apostas,1,0,0,0,1
Assinaturas e Servicos,3,21,1,4,0
Bar,1,4,7,0,0
Beleza,12,6,5,2,2
Casa,2,6,12,3,0
Faculdade,1,1,4,1,0
Ifood,30,36,30,32,24
Lanches e Conveniencias,1,7,7,4,0
Lavanderia,16,12,15,8,0


In [62]:
from scipy.stats import chi2_contingency

chi2_contingency(tabela)

stat, p, dof, expected = chi2_contingency(tabela)

print(f'estatística do teste: {stat:.4f}')
print(f'p-valor: {p}')
print(f'graus de liberdade: {dof}')

estatística do teste: 233.6331
p-valor: 4.9932050210422784e-20
graus de liberdade: 68


In [63]:
# Decisão

alfa = 0.05

if p < alfa:
  print('Hipótese nula rejeitada')
  print('Há relação entra as variáveis')
else:
  print('Hipótese nula não rejeitada')
  print('Não há relação entre as variáveis')

Hipótese nula rejeitada
Há relação entra as variáveis


In [65]:
df_valor_esperado = pd.DataFrame(expected, index=tabela.index, columns=tabela.columns)

df_valor_esperado.head()

Semana Mes,1,2,3,4,5
Categoria,,,,,
Apostas,0.528033,0.499490,0.495413,0.362895,0.114169
Assinaturas e Servicos,7.656473,7.242610,7.183486,5.261978,1.655454
Bar,3.168196,2.996942,2.972477,2.177370,0.685015
Beleza,7.128440,6.743119,6.688073,4.899083,1.541284
Casa,6.072375,5.744139,5.697248,4.173293,1.312946


In [69]:
# Onde está a relação de dependência?

residuos_de_pearson = (tabela - expected) / np.sqrt(expected)
residuos_de_pearson.head()

Semana Mes,1,2,3,4,5
Categoria,,,,,
Apostas,0.649504,-0.706746,-0.703856,-0.602408,2.621659
Assinaturas e Servicos,-1.682840,5.111973,-2.307097,-0.550145,-1.286644
Bar,-1.218127,0.579411,2.336032,-1.475591,-0.827657
Beleza,1.824613,-0.286173,-0.652741,-1.309795,0.369490
Casa,-1.652603,0.106756,2.640570,-0.574337,-1.145839


In [70]:
# Relações significativas: >2 ou <-2

mask = abs(residuos_de_pearson)> 2
significativos = residuos_de_pearson[mask]
significativos

Semana Mes,1,2,3,4,5
Categoria,,,,,
Apostas,NaN,NaN,NaN,NaN,2.621659
Assinaturas e Servicos,NaN,5.111973,-2.307097,NaN,NaN
Bar,NaN,NaN,2.336032,NaN,NaN
Beleza,NaN,NaN,NaN,NaN,NaN
Casa,NaN,NaN,2.640570,NaN,NaN
Faculdade,NaN,NaN,NaN,NaN,NaN
Ifood,NaN,NaN,NaN,NaN,5.201953
Lanches e Conveniencias,NaN,NaN,NaN,NaN,NaN
Lavanderia,NaN,NaN,NaN,NaN,NaN
